# Introductory machine learning experiment with Scikit-Learn

A tutorial to showcase a basic machine learning pipeline that you can apply to your own tabular datasets.

## Google colab setup

This cell downloads and installs the notebook's requirements in the Google Colab runtime.
You should comment or delete it if you are not running the notebook on Google Colab.

In [ ]:
import sys
import os

# Download the repository from GitHub
!git clone https://github.com/5TuX/ml-radiomics-classification.git

# Set working directory
%cd ml-radiomics-classification

# Install uv dependency manager
!curl -LsSf https://astral.sh/uv/install.sh | sh

# Sync dependencies with uv
!uv sync

# Add src directory to Python path so modules can be imported
sys.path.insert(0, os.path.join(os.getcwd(), "src"))

## Python imports and settings

In [ ]:
# Python imports
import re
import platform
from itertools import groupby

# Data handling imports
import polars as pl

# Scikit-learn utility imports
import sklearn
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_validate, train_test_split, RandomizedSearchCV
from sklearn.metrics import (
    balanced_accuracy_score,
    classification_report,
    ConfusionMatrixDisplay,
)

# Scikit-learn model imports
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

# Other imports
import numpy as np
import joblib
from tabulate import tabulate  # for pretty-printing tables
import altair as alt  # for pretty visualizations

# Configuration settings
settings = {
    "dataset-path": "data/Radiomics_binWidth-15_ZScore_NETnNCR_T1CE.csv",  # our dataset file
    "random_state": 42,  # set a seed to ensure reproducibility of random procedures
}

## Introduction

Let's imagine we have a reasonably sized **tabular dataset** (e.g. a CSV file) where each line is a sample and each column is a feature.

We want to **train** a model to **predict** one of these columns (the target feature) using the other columns (the input features).

### The dataset

In this notebook, we are looking at a version of the [BraTS-2020-Openradiomics](https://openradiomics.org/?page_id=1087) dataset (see [publication](https://doi.org/10.1186/s12880-025-01855-2)) whic contains information of 369 adult patients with brain tumor.

<figure style="margin: auto; text-align: left; max-width:300px;">
    <img src="https://media.springernature.com/full/springer-static/image/art%3A10.1186%2Fs12880-025-01855-2/MediaObjects/12880_2025_1855_Fig1_HTML.png" style="max-width:300px;">
    <figcaption>An example BraTS 2020 image (the FLAIR sequence) and its corresponding segmentation mask. The orange area is AT (active tumor), the green area is ED (peritumoral edematous/invaded tissue), and the gray parts are NETnNCR (Union of necrotic and non-enhancing tumor)</figcaption>
</figure>

Each line contains (among other things):
- Around 1500 [radiomic](https://en.wikipedia.org/wiki/Radiomics) features computed from tumor segmentations of a patient's MRI scan (computation was done using the [PyRadiomics](https://pyradiomics.readthedocs.io/) library). These features provide shape, color and texture information about the tumor. The original MRI images are *not* included.
- The category of brain tumor: "LGG" for low-grade glioma (less aggressive tumor) or "HGG" for high-grade glioma (more aggressive tumor)
- Survival information (in days, provided for 236 patients)

**Goal:** we will train a binary classifier to predict the class of the tumor (LGG or HGG) from the radiomics features (we'll ignore the survival information entirely).

### Training machine learning models

There are a lot of general approaches to machine learning:
- Supervised learning trains models on labelled data (most common)
- Unsupervised learning searches patterns in unlabelled data
- Self-supervised learning uses labels automatically generated from the data itself (e.g., contrastive learning)
- Reinforcement learning learns by interacting in an environment to maximize a reward
- Many more learning paradigms are listed on wikipedia: https://en.wikipedia.org/wiki/Machine_learning

In this notebook, we'll focus on a **supervised classification task**, which consists in predicting a class label from a set of input variables. *(Another common supervised machine learning task is regression, which tries to predict a continuous value. It won't be addressed here.)*

The Python **scikit-learn** library provides lots of tools to experiment with machine learning methods on our dataset. In this tutorial, we will showcase some of these tools that will help us to:
- **Split** the dataset in training and test subset (crucial to detect overfitting)
- Create a **preprocessing** pipeline for the data (category encoding, variable scaling)
- Leverage **cross-validation** to quickly compare a bunch of different models
- Do the final **training** and **evaluation** of one selected model

### Spending time with the data

Playing around with AI models is only a small part of the job. Most of the time is spent with the dataset to make sure what we do has a meaning. This involves:
- loading the data
- visualizing and exploring the data
- looking at feature distributions and correlations
- selecting input and target features
- handling missing values
- possibly creating new features
- ...

In this tutorial, we'll hide most of this part to focus on scikit-learn, but keep in mind that in a real-world scenario your dataset will never be ready to use for training out-of-the box.

A widely used Python library for handling tabular data is Pandas. In this tutorial however, we prefered to briefly showcase a more recent library named [Polars](https://github.com/pola-rs/polars). Notably, Polars prefers to use [Altair](https://github.com/vega/altair) over Matplotlib for interactive plotting and visualization, which we will also use to gain some visual insights. Polars provides tools to convert to Pandas format, but scikit-learn is compatible with both libraries anyway.

## Phase 1 - Preparing the data

Here we decide what columns to keep, how to handle missing values, and we put a test set aside (20% of the data) for evaluating our final model. All further data pre-processing and model training will be done on the rest of the data without ever looking at the test set.

To do all this, we'll use the Polars library (Pandas is also a fine choice).

### Loading the data

Let's load the dataset from the CSV file and look at a few raw examples to get an idea of the data structure:

In [ ]:
df = pl.read_csv(settings["dataset-path"])
print("Dataset shape:", df.shape)
df.head()

### Looking at some basic statistics

The `describe()` method computes basic information about the dataset:

- Variable means, standard deviations etc.
- Number undefined values in each column:
    - Age and survival days are available for only 133 out of 369 patients
    - One row seems to have all radiomic features missing

In [ ]:
df.describe()

### Making sense of the columns structure

There is a huge number of columns in this dataset. Let's quickly explore the structure of column names. In the Pyradiomics documentation, we can learn more about all [radiomics features](https://pyradiomics.readthedocs.io/en/latest/features.html) and the different [image filters](https://pyradiomics.readthedocs.io/en/latest/customization.html#image-types) that can be applied before computation.

In [ ]:
print(f"{len(df.columns)} columns in total.\n")


def key(colname):
    return colname.split("_")[0]


for column_group, columns in groupby(df.columns, key):
    print(column_group)
    columns = list(columns)
    if len(columns) == 1:
        print(f"\t-> {columns}")
        continue
    columns = [colname[len(column_group) + 1 :] for colname in columns]
    for column_subgroup, subcolumns in groupby(columns, key):
        maxlen, suffix = 3, ""
        subcolumns = [colname[len(column_subgroup) + 1 :] for colname in subcolumns]
        if (lensubcols := len(subcolumns)) > maxlen:
            subcolumns = subcolumns[:maxlen]
            suffix = f"... ({lensubcols} total)"
        print(f"\t{column_subgroup} -> {subcolumns} {suffix}")

### Selecting columns to keep

We can either specify columns to drop or to keep.
In this case, there are 1720 columns, so it's easier to specify which ones to keep.

- We set the "Group" feature as our target variable
- We use all original radiomics (columns with name starting with "original_") as input features

In [ ]:
# Target variables:
# - ignore Survival_days (too many missing values)
# - ignore Group_label (redundant with Group)
targets = ["Group"]

# Clinical inputs:
# - ignore Age (many missing values)
# - ignore Extent_of_Resection (many missing values)
inputs_clinical = []

# Radiomic inputs:
# - ignore all derived features (columns not starting with "original_")
inputs_radiomics = [col for col in df.columns if col.startswith("original_")]
df = df.select(targets + inputs_clinical + inputs_radiomics).rename(
    {col: col[9:] for col in inputs_radiomics}
)

# Also ignored:
# - Patient_ID
# - binWidth, Normalization, Subregion, Sequence (describes the data was used to compute the radiomics)
# - all columns starting with "diagnostics_" (describes the radiomic computation process)

df.head()

### Handling missing values

Generally, there are different strategies to handle missing data:
- Dropping all rows with missing values (like we do here since there is only one)
- Dropping columns with missing values (after checking there is almost no data in them)
- Filling missing values, e.g. with the median (imputation)

In this case we have seen that there is only one row with missing data, so we'll just get rid of it.

Then we'll have a final look at our prepared data:

In [ ]:
df = df.drop_nulls()
print("Dataset shape after dropping nulls:", df.shape)

df.describe()

### Creating a test set

It is important to set part of the data aside to make sure our final model performs well on unseen data.

- Here we use a stratified split to make ensure that training and test sets have the same class distribution.
- We also specify a random seed to ensure reproducibility of the sample shuffling.

In [ ]:
training_data, test_data = train_test_split(
    df,
    test_size=0.20,
    stratify=df["Group"],
    random_state=settings["random_state"],
)


def check_class_proportions(df: pl.DataFrame):
    summary = (
        df.group_by("Group")
        .agg(pl.len().alias("Count"))
        .with_columns((pl.col("Count") / pl.col("Count").sum()).alias("Proportion"))
        .sort("Count", descending=True)
    )
    with pl.Config(set_tbl_hide_column_data_types=True, tbl_hide_dataframe_shape=True):
        print(summary)


print(f"\nTraining dataset ({len(training_data)} samples):")
check_class_proportions(training_data)

print(f"\nTest dataset: ({len(test_data)} samples)")
check_class_proportions(test_data)

## Phase 2 - Pre-processing the training data

The data needs to be pre-preocessed before feeding it to machine learning algorithms:

Scikit-learn provides [transformer](https://scikit-learn.org/stable/data_transforms.html) objects to customize your pipelines (unrelated to the famous deep learning transformer architecture). To learn more about the thinking behind this, you can have a look at the library's [design principles](https://doi.org/10.48550/arXiv.1309.0238).

Each transformer has two important methods:
- `fit` is used a single time to record what preprocessing needs to be done
- `transform` can be used any number of times in downstream tasks to apply the pre-processing to new inputs.

We'll also use the `Pipeline` and `ColumnTransformer` classes to wrap every transformation in a single final pipeline that we can later apply to any input data.

With this slightly complicated but powerful set of tools, we can build virtually any pre-processing pipeline.

**Warning**: some pre-processing functions depend on data statistics, so it is crucial to use *only training data* at this stage.

### Separating input and target variables

Data preprocessing is typically applied to input features only, so let's start by separating input and target variables:

In [ ]:
training_inputs_df = training_data.drop("Group")
training_targets_df = training_data.select("Group")

print(f"{type(training_inputs_df)=}")
print(f"{type(training_targets_df)=}")

### Encoding the target variables

Each sample has a target class label to predict, either "HGG" or "LGG".

Scikit-learn provides a `LabelEncoder` class that converts label strings to label integer IDs. It can be fitted to the target data and later reused to convert the output of the model (class 0, class 1...) to a human-readable format ("HGG", "LGG").

In [ ]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(training_targets_df.to_numpy().ravel())

print(f"{type(y_train) =}, {y_train.shape = }\n")
print(f"{y_train[:10] = }\n")

for i, class_name in enumerate(label_encoder.classes_):
    class_support = sum(y_train == i)
    print(f"Class {i}: {class_name} (support: {class_support})")

### Pre-processing categorical input features

Similar to how we encoded target class labels, we also want to use a proper encoding for **categorical input features**: these are originally string values that initially don't mean anything to AI models. They need to be converted to numbers first.

In a lot of cases, categorical variables ("Blue", "Red", "Green") have no order. A practical way of representing them is through **one-hot encoding**: we create a binary variable for each possible category, and set all values to zero except for the correct categories: "Blue" becomes (1, 0, 0), "Red" becomes (0, 1, 0), "Green" becomes (0, 0, 1).

Scikit-learn provides a `OneHotEncoder` class that can be fitted to our data to convert categorical string data to binary numbers.

In this notebook, there are no categorical input variables, but we still write a code to handle them in case the data changes at some point:

In [ ]:
categorical_features = training_inputs_df.select(pl.col(pl.Utf8)).columns

print(f"Number of categorical features: {len(categorical_features)}")

# Define a Pipeline with a single "onehot" step:

categorical_pipeline = Pipeline(
    [
        ("onehot", OneHotEncoder()),
    ]
)
categorical_pipeline

### Pre-processing numerical input features

Most machine learning models don't perform well on numerical features that have widly different scales (some models work perfectly fine with un-normalized data, like the powerful random forest).

It therefore is a good practice to apply **feature scaling** methods such as *min-max scaling* and *standardization* (also known as *Z-Score normalization*). Here, we'll go for the latter.

Scikit-learn provides a `StandardScaler` class that remembers the mean and variance of the input feaatures and can later be used later to normalize them. (fit this only on training data!)

In [ ]:
# Handling numerical input variables (feature scaling):

numerical_features = training_inputs_df.select(pl.col(pl.Float64)).columns

print(f"Number of numerical features: {len(numerical_features)}")

numerical_pipeline = Pipeline(
    [
        ("scaler", StandardScaler()),
    ]
)
numerical_pipeline

### Defining a complete pre-processing pipeline

Once all necessary transformations are defined and wrapped in their respective `Pipeline` objects, we wrap everything in a `ColumnTransformer` that will apply each transformation to the appropriate subset of features.

Once that's done, we can fit this big pipeline to our training data and apply the transformation. The `fit_transform` method is a convenient shortcut to fit the pipeline on the data and apply the transformation in a single line of code:

In [ ]:
# Define our final pre-processing pipeline:

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_pipeline, numerical_features),
        ("cat", categorical_pipeline, categorical_features),
    ]
)

# Fit on training data and run pre-processing

X_train = preprocessor.fit_transform(training_inputs_df)

print(f"{type(X_train) = }, {X_train.shape = }")

### Final look at the pre-processed training data

Let's look at one feature distribution before and after pre-processing and check that the average value is now zero:

In [ ]:
# Convert the preprocessed data back to a DataFrame for easier visualization

feature_names = preprocessor.get_feature_names_out().tolist()
simple_feature_names = [re.sub(r"^.*?__", "", name) for name in feature_names]
X_train_df = pl.DataFrame(X_train, schema=simple_feature_names)

# Look at some preprocessed input features distributions:

max_features_to_plot = 1

print("Before preprocessing:")
for feature in training_inputs_df.columns[:max_features_to_plot]:
    training_inputs_df[feature].plot.hist().show()

print("After preprocessing:")
for feature in X_train_df.columns[:max_features_to_plot]:
    X_train_df[feature].plot.hist().show()

## Phase 3 - Cross-validating a few models

Now that our data is correctly pre-processed, we can finally start training some models. Since our dataset is small, we can test multiple models in a reasonable amount of time and we can use cross-validation instead of using a regular train/validation split. What does this mean?

In a normal train/validation split, we separate part of the training data for validation to estimate how well the model will generalize to unseen data. This helps detect [overfitting](https://scikit-learn.org/stable/auto_examples/model_selection/plot_underfitting_overfitting.html), where a model performs well on training data but poorly on new data. However, a single split can give misleading results if the partition is unbalanced or too small. Cross-validation addresses this by training and evaluating the model multiple times on different folds of the data, ensuring every sample is used for validation once. This provides a more robust and reliable estimate of the model’s true performance while still guarding against overfitting. At the end, we obtain multiple values for our performance metric and typically look at their average.

Once cross-validation is complete and a model is selected, we retrain it on the entire training data (both train and validation). Finally, we evaluate it on the test set that was set aside at the beginning and never used during training or model selection.

#### List classification models that we want to compare

Here we instantiate some classification models we'd like to try on our training data. This list was essentially taken from scikit-learn documentation's [classifier comparison tutorial](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html).

We set a random seed for reproducibility.

In this tutorial, we defined two lists of models, one with default parameters and the other with slightly tweaked settings.

**Note:** The [MLPClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier) class is an example of a deep learning model. However scikit-learn's implementation is meant for leightweight use only and doesn't include GPU support. To take things to the next level, you should use a library dedicated to deep learning such as [Pytorch](https://github.com/pytorch/pytorch).


In [ ]:
rnd_state = 42  # set to None for a different random state each time

# First list of models with default parameters
models1 = {
    "Logistic Regression": LogisticRegression(random_state=settings["random_state"]),
    "Decision Tree": DecisionTreeClassifier(random_state=settings["random_state"]),
    "Random Forest": RandomForestClassifier(random_state=settings["random_state"]),
    "AdaBoost": AdaBoostClassifier(random_state=settings["random_state"]),
    "Linear SVM": SVC(random_state=settings["random_state"]),
    "RBF SVM": SVC(random_state=settings["random_state"]),
    "Neural Net": MLPClassifier(random_state=settings["random_state"]),
    "Nearest Neighbors": KNeighborsClassifier(),
    "Naive Bayes": GaussianNB(),
}

# Second list of models with a bit more customization
models2 = {
    "Logistic Regression": LogisticRegression(
        random_state=settings["random_state"], C=0.5
    ),
    "Decision Tree": DecisionTreeClassifier(
        max_depth=3, random_state=settings["random_state"]
    ),
    "Random Forest": RandomForestClassifier(
        n_estimators=50, max_depth=5, random_state=settings["random_state"]
    ),
    "AdaBoost": AdaBoostClassifier(
        learning_rate=0.6, random_state=settings["random_state"]
    ),
    "Linear SVM": SVC(kernel="linear", C=0.025, random_state=settings["random_state"]),
    "RBF SVM": SVC(gamma="scale", C=1, random_state=settings["random_state"]),
    "Neural Net": MLPClassifier(
        alpha=1, max_iter=1000, random_state=settings["random_state"]
    ),
    "Nearest Neighbors": KNeighborsClassifier(n_neighbors=3),
    "Naive Bayes": GaussianNB(var_smoothing=1e-10),
}

#### Decide on a performance metric for model evaluation

There are many performance metrics that we can use to evaluate and compare models. Some of the most commons are: accuracy, class recall, class precision, class f1-score. Scikit-learn's documentation lists a huge variety of other metrics that can be used:: https://scikit-learn.org/stable/modules/model_evaluation.html

Scikit-learn does support custom performance metric definition, but in this notebook, we're going to keep things simple and use a *balanced accuracy score*, since we don't have the same number of samples in each class.

#### Run cross-validation for each model

Here we use scikit-learn's `cross-validate` method that returns training and validation scores for each validation split (5 total). We store the average and standard deviation of each score for later comparison. It is important that we do keep both training and validation scores, as the comparison between both allows us to spot oeverfitting issues.

In [ ]:
def cross_validate_models(models, X_train, y_train):
    results = []

    scoring = "balanced_accuracy"

    for model_name, model in models.items():
        print(f"\nTrying model: {model_name}")
        cv_results = cross_validate(
            model,
            X_train,
            y_train,
            cv=5,
            scoring=scoring,
            return_train_score=True,
        )

        train_mean = cv_results["train_score"].mean()
        train_std = cv_results["train_score"].std()
        val_mean = cv_results["test_score"].mean()
        val_std = cv_results["test_score"].std()

        results.append(
            {
                "Model": model_name,
                "Train Mean": train_mean,
                "Train Std": train_std,
                "Val Mean": val_mean,
                "Val Std": val_std,
            }
        )

        print(f"Train {scoring}: {train_mean:.4f} ± {train_std:.4f}")
        print(f"Validation {scoring}: {val_mean:.4f} ± {val_std:.4f}")
        print(f"Train vs Val difference: {train_mean - val_mean:.4f}")
    return results


results1 = cross_validate_models(models1, X_train, y_train)

print("\n" + "=" * 100 + "\n")
results2 = cross_validate_models(models2, X_train, y_train)

#### Look at the results and compare the models

We then sort the models by average validation score to see which ones perform better.

We notice that some models from the first list have a bigger performance difference (above 0.1) between training and validation data, and sometimes a perfect train score. These are signs of overfitting. Ideally, train and validation score should be both high and close to each other.

Results from the second models list with tweaked parameters are better, the difference between train and validation scores is lower.

Overall, it seems that AdaBoost, Random Forest and Neural Net are the most promising models in this case.

In [ ]:
def list_results_sorted(results):
    # Sort by validation mean
    results = sorted(results, key=lambda x: x["Val Mean"], reverse=True)

    # Prepare table for tabulate (remove Val Mean from display)

    table = []
    for res in results:
        train_val_diff = res["Train Mean"] - res["Val Mean"]
        table.append(
            [
                res["Model"],
                f"{res['Train Mean']:.3f} ± {res['Train Std']:.3f}",
                f"{res['Val Mean']:.3f} ± {res['Val Std']:.3f}",
                train_val_diff,
            ]
        )

    print(
        tabulate(
            table,
            headers=["Model", "Train Score", "Validation Score", "Train-val Diff"],
            tablefmt="fancy_grid",
        )
    )


print("\nResults for models1:")
list_results_sorted(results1)

print("\nResults for models2:")
list_results_sorted(results2)

## Phase 4 - Fine-tuning a chosen model

Looking at the above model comparison, we choose to carry on with the `AdaBoostClassifier` model.

Now we want to find the best possible parameters to get the best performance out of this model.

Looking at the documentation, we spot two hyperparameters that we can play with: `learning_rate` and `n_estimators` (learn more in the documentation [link text](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html))

### Searching for the best set of model hyperparameters

Scikit-learn provides tools to automatically train and evaluate models with different hyperparameters. We can use a **grid search**, which tests all parameter combinations, or a **random search**, which samples a subset at random.

Here we'll go for a random search, since we want to explore a large space but we don't have the time to try every combination.

**Note:** after finishing the search, scikit-learn automatically *refits* a model with the best hyperparameters on the entire training set. This final model can be access through the random search's `_best_estimator` attribute.

In [ ]:
# Define the hyperparameter space to explore
param_space = {
    "learning_rate": [
        0.1,
        0.2,
        0.3,
        0.4,
        0.5,
        0.6,
        0.7,
        0.8,
        0.9,
        1,
        1.1,
        1.2,
        1.3,
        1.4,
        1.5,
    ],
    "n_estimators": [1, 15, 50, 100, 150, 200, 250, 300],
}

# You can also define hyperparameter distributions like this:
# from scipy.stats import randint, uniform
# lr_min, lr_max = 0.1, 1.5
# learning_rate_distrib = uniform(loc=lr_min, scale=lr_max-lr_min)
# n_estimators_distrib = randint(1, 300)

# Define and run the randoms search
random_search = RandomizedSearchCV(
    AdaBoostClassifier(random_state=settings["random_state"]),
    param_distributions=param_space,
    n_iter=150,
    scoring="balanced_accuracy",
    cv=5,
    random_state=settings["random_state"],
    n_jobs=2,
    verbose=2,
)
random_search.fit(X_train, y_train)

# Print search results sorted by performance on validation data
cv_res = pl.DataFrame(random_search.cv_results_, strict=False)
cv_res = cv_res.sort("mean_test_score", descending=True).select(
    [
        col
        for col in cv_res.columns
        if not (col.startswith("split") or col.endswith("_time") or col == "params")
    ]
)
print(cv_res)

# Retrieve the best model
print("Best model:")
best_model = random_search.best_estimator_
best_model

### Looking for patterns in the hyperparameter space (bonus)

Let's create a plot to visualize the relationship between the model's hyperparameters and the performance metric.

Here we showcase a use of the [Altair](https://github.com/vega/altair) library that is great for interative visualization: you can zoom on the plot and hover your mouse over dots to see their detailed information.

*(Coding nice plots can be daunting. This code snippet was generated entirely using ChatGPT)*

In [ ]:
min_score = cv_res["mean_test_score"].min()
max_score = cv_res["mean_test_score"].max()
score_range = max_score - min_score if max_score != min_score else 1.0

chart = (
    alt.Chart(cv_res)
    .transform_calculate(
        # normalized version (0–1)
        score_norm=f"(datum.mean_test_score - {min_score}) / {score_range}",
        # exponential scaling for size (power of 3)
        size_exp=f"pow((datum.mean_test_score - {min_score}) / {score_range}, 3)",
    )
    .mark_circle()
    .encode(
        x="param_n_estimators",
        y="param_learning_rate",
        color=alt.Color(
            "mean_test_score:Q",
            scale=alt.Scale(
                domain=[min_score, max_score],
                scheme="turbo",  # bright and high contrast
                clamp=True,
            ),
            legend=alt.Legend(title="Mean Test Score"),
        ),
        size=alt.Size(
            "size_exp:Q",
            scale=alt.Scale(range=[80, 900]),
            legend=None,
        ),
        tooltip=["param_n_estimators", "param_learning_rate", "mean_test_score"],
    )
    .properties(
        title="AdaBoost Hyperparameter Search Results",
        width=600,
        height=400,
    )
    .interactive()
)
chart.show()

### Testing the final model on the test set

Let's do a final evaluation of our model. For this we'll go back to the test data that we set aside at the very beginning.

We won't be surprised if the performance is slightly worse since this data was never seen during training.

In [ ]:
# Test the best model on the held-out test dataset

test_inputs_df = test_data.drop("Group")
test_targets_df = test_data.select("Group")
X_test = preprocessor.transform(test_inputs_df)
y_test = label_encoder.transform(test_targets_df.to_numpy().ravel())

y_pred = best_model.predict(X_test)
print("Balanced Accuracy on test data:", balanced_accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))
print("\nConfusion Matrix:")
ConfusionMatrixDisplay.from_predictions(
    y_test, y_pred, display_labels=label_encoder.classes_
)

### Exporting the final pipeline

We need a way to persist our pre-processing functions and trained model for future use without having to retrain. An easy way to achieve this is to use the joblib library. Other methods of [model persistance](https://scikit-learn.org/stable/model_persistence.html#pickle-joblib-and-cloudpickle) are listed in scikit-learn's documentation.

In [ ]:
# Save pre-processing functions and model for alter use
model_bundle = {
    "artifacts": {
        "label_encoder": label_encoder,
        "pipeline": Pipeline(
            [
                ("preprocessor", preprocessor),
                ("model", best_model),
            ]
        ),
    },
    "metadata": {
        "python_version": sys.version,
        "platform": platform.platform(),
        "sklearn_version": sklearn.__version__,
        "numpy_version": np.__version__,
    },
}
joblib.dump(model_bundle, "model.joblib")

# Load the saved pre-processing and model
loaded_model = joblib.load("model.joblib")

# Run and evaluate the loaded model on test data
test_pred = loaded_model["artifacts"]["pipeline"].predict(test_inputs_df)
test_pred_labels = loaded_model["artifacts"]["label_encoder"].inverse_transform(
    test_pred
)

print(
    classification_report(
        test_targets_df, test_pred_labels, target_names=label_encoder.classes_
    )
)
print(
    "Balanced Accuracy on test data:", balanced_accuracy_score(y_test, test_pred), "\n"
)

print(
    tabulate(
        [
            ["Prediction", str(test_pred_labels[5:15])],
            ["Expected", str(test_targets_df.to_numpy().ravel()[5:15])],
        ],
        tablefmt="fancy_grid",
    )
)

## Going further

- This notebook does not cover **feature selection**, which is the process of identifying the most relevant variables for a model. Scikit-learn provides various tools for this purpose. More details are available in the [official documentation](https://scikit-learn.org/stable/modules/feature_selection.html).

- There are specialized **automated machine learning** (AutoML) libraries for fast model comparison. You can have a look at [pycaret](https://github.com/pycaret/pycaret), [lazypredict](https://github.com/shankarpandala/lazypredict) or [tpot](https://github.com/EpistasisLab/tpot)

- For more powerful model **hyperparameter fine-tuning**, you can have a look at a dedicated framework like [optuna](https://github.com/optuna/optuna)

## References

- This notebook is heavily inspired by Aurelien Geron's [End-to-end machine learning project](https://github.com/ageron/handson-ml3/blob/main/02_end_to_end_machine_learning_project.ipynb) notebook from his book [Hand-on machine learning with scikit-learn and Pytorch](https://www.oreilly.com/library/view/hands-on-machine-learning/9798341607972/). In this tutorial we're looking at a classification task, but his example focuses on regression. Feel free to check it out if you are interested.
- Andrej Karpathy provides a nifty [Recipe for Training Neural Networks](https://karpathy.github.io/2019/04/25/recipe/) which gives useful guidelines for any machine learning project and points at common mistakes to avoid.
- The CNRS has a freely accessible program called [FIDLE](https://fidle.cnrs.fr), yearly revisited, wich provides a nice entry point for anyone curious about artifical intelligence.